In [15]:
import pandas as pd
from pycaret.regression import *

df = pd.read_csv("../../data/full_dataset.csv")

df = df.drop(columns=["Unnamed: 0", "name"], errors='ignore')
df

,price,location,bedroom,bathroom,land_area,building_area
0,1680.0,"Rungkut, Surabaya",3,3,67,120
1,950.0,"Medokan Ayu, Surabaya",3,3,72,140
2,500.0,"Wonorejo, Surabaya",2,1,30,36
3,950.0,"Sukolilo, Surabaya",2,2,50,55
4,600.0,"Kebraon, Surabaya",2,1,50,56
...,...,...,...,...,...,...
32784,2500.0,Surabaya,4,4,150,200
32785,2500.0,"Sukolilo, Surabaya",4,2,147,200
32786,2500.0,Surabaya,3,2,240,300
32787,2500.0,Surabaya,2,1,105,125


In [16]:
df.duplicated().sum()

13086

In [17]:
df = df.drop_duplicates()

In [18]:
df.duplicated().sum()

0

In [21]:
lokasi_count = df['location'].value_counts()
lokasi_count

location
Rungkut, Surabaya         2617
Surabaya                  2284
Sidoarjo, Sidoarjo        1690
Wiyung, Surabaya          1103
Kenjeran, Surabaya         904
                          ... 
Candi, Sidoarjo              1
Karangploso, Malang          1
Kedamean, Gresik             1
Kedungkandang, Malang        1
Balongpanggang, Gresik       1
Name: count, Length: 143, dtype: int64

In [22]:
lokasi_kecil = lokasi_count[lokasi_count < 10].index
df['location'] = df['location'].apply(lambda x: 'Lainnya' if x in lokasi_kecil else x)
(df['location'] == 'Lainnya').value_counts()

location
False    19510
True       193
Name: count, dtype: int64

In [19]:
# import ydata_profiling as pp

# profile = pp.ProfileReport(df)
# profile.to_file('pp_report/Report_full.html')

In [23]:
setup(
    df, 
    target='price',
    transform_target=True,
    normalize=True, 
    normalize_method='minmax',
    remove_outliers=True, 
    outliers_threshold=0.05,
    categorical_features=['location'],
    session_id=42
)

,Description,Value
0,Session id,42
1,Target,price
2,Target type,Regression
3,Original data shape,"(19703, 6)"
4,Transformed data shape,"(19013, 6)"
5,Transformed train set shape,"(13102, 6)"
6,Transformed test set shape,"(5911, 6)"
7,Numeric features,4
8,Categorical features,1
9,Rows with missing values,0.0%


In [24]:
# best = compare_models()
top3_models = compare_models(n_select=3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,236.3280,98256.4013,313.3677,0.6399,0.2765,0.2238,0.2860
catboost,CatBoost Regressor,236.0039,98481.2844,313.7221,0.6392,0.2748,0.2210,1.7000
xgboost,Extreme Gradient Boosting,233.1220,99806.2906,315.7881,0.6344,0.2766,0.2170,0.1590
rf,Random Forest Regressor,232.6627,101354.1266,318.2490,0.6286,0.2772,0.2170,0.5820
gbr,Gradient Boosting Regressor,248.8992,106725.7101,326.6057,0.6088,0.2879,0.2367,0.3200
et,Extra Trees Regressor,237.3141,106822.9431,326.7096,0.6085,0.2853,0.2211,0.6680
knn,K Neighbors Regressor,257.9701,118404.8477,344.0090,0.5661,0.3004,0.2434,0.0710
ada,AdaBoost Regressor,288.4716,130359.0738,360.9687,0.5221,0.3225,0.2809,0.1640
dt,Decision Tree Regressor,287.2961,167813.2214,409.4155,0.3850,0.3558,0.2598,0.0660
omp,Orthogonal Matching Pursuit,368.0737,210765.1619,458.8993,0.2280,0.4084,0.3740,0.0650


In [25]:
blended = blend_models(estimator_list=top3_models)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,227.0783,90415.4466,300.6916,0.6726,0.2678,0.2149
1,230.5676,94765.1342,307.8395,0.6503,0.2747,0.2164
2,226.4505,88145.1262,296.8924,0.6571,0.2525,0.2019
3,224.4066,91486.3914,302.4672,0.6652,0.2653,0.2100
4,233.7326,98008.6537,313.0633,0.6314,0.2802,0.2252
5,235.7242,97452.0216,312.1731,0.6440,0.2704,0.2194
6,227.9158,91796.3942,302.9792,0.6545,0.2665,0.2145
7,234.8704,97514.4222,312.2730,0.6558,0.2715,0.2202
8,236.1663,102445.3432,320.0708,0.6504,0.2785,0.2216


In [26]:
predict_model(blended)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,228.8412,93052.3995,305.0449,0.6549,0.2677,0.2156


,location,bedroom,bathroom,land_area,building_area,price,prediction_label
17201,"Mulyosari, Surabaya",3,3,122,144,1950.0,1832.384091
11759,"Rungkut, Surabaya",2,1,105,90,975.0,1004.513850
4043,"Sambikerep, Surabaya",5,4,139,150,1750.0,1707.789072
22778,"Simokerto, Surabaya",1,1,93,90,1030.0,778.703591
2127,"Kebraon, Surabaya",3,2,84,110,1700.0,1358.827958
...,...,...,...,...,...,...,...
6807,"Darmo Permai, Surabaya",2,2,78,85,950.0,1403.803089
12515,"Sidoarjo, Sidoarjo",2,1,72,45,438.0,547.284628
18748,"Gununganyar, Surabaya",3,2,75,90,1600.0,1205.837734
7821,"Surabaya Kota, Surabaya",2,2,90,85,2000.0,1262.625540


In [27]:
stacked = stack_models(estimator_list=top3_models)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,226.2107,90391.4240,300.6517,0.6727,0.2668,0.2131
1,230.9665,96681.7846,310.9369,0.6432,0.2778,0.2158
2,226.2027,88314.2872,297.1772,0.6564,0.2518,0.2005
3,223.5560,91074.7509,301.7859,0.6667,0.2639,0.2082
4,233.1827,97762.8488,312.6705,0.6323,0.2795,0.2243
5,235.4984,97997.0774,313.0448,0.6420,0.2707,0.2191
6,227.5048,92292.3553,303.7966,0.6526,0.2658,0.2129
7,235.2599,98574.5393,313.9658,0.6521,0.2723,0.2198
8,236.5196,103276.6757,321.3669,0.6475,0.2785,0.2208


In [28]:
predict_model(stacked)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Stacking Regressor,228.1456,93423.1749,305.6520,0.6535,0.2674,0.2142


,location,bedroom,bathroom,land_area,building_area,price,prediction_label
17201,"Mulyosari, Surabaya",3,3,122,144,1950.0,1823.275036
11759,"Rungkut, Surabaya",2,1,105,90,975.0,1003.945989
4043,"Sambikerep, Surabaya",5,4,139,150,1750.0,1718.028842
22778,"Simokerto, Surabaya",1,1,93,90,1030.0,772.913164
2127,"Kebraon, Surabaya",3,2,84,110,1700.0,1384.298873
...,...,...,...,...,...,...,...
6807,"Darmo Permai, Surabaya",2,2,78,85,950.0,1389.938414
12515,"Sidoarjo, Sidoarjo",2,1,72,45,438.0,552.864041
18748,"Gununganyar, Surabaya",3,2,75,90,1600.0,1222.818749
7821,"Surabaya Kota, Surabaya",2,2,90,85,2000.0,1270.372296


In [29]:
tuned_model = tune_model(blended, search_library='scikit-optimize', search_algorithm='bayesian')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,226.7620,90252.6300,300.4208,0.6732,0.2681,0.2151
1,230.3165,93688.4632,306.0857,0.6543,0.2734,0.2165
2,226.1759,87873.4132,296.4345,0.6581,0.2531,0.2023
3,224.9857,92397.7869,303.9700,0.6619,0.2669,0.2113
4,233.7632,98234.7667,313.4243,0.6305,0.2804,0.2253
5,236.1003,97782.5177,312.7020,0.6428,0.2707,0.2197
6,227.9947,91623.5588,302.6938,0.6552,0.2668,0.2149
7,233.8100,96700.3838,310.9669,0.6587,0.2707,0.2198
8,235.8236,102329.4552,319.8898,0.6508,0.2788,0.2221


Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits


In [31]:
tuned_model = tune_model(stacked, search_library='scikit-optimize', search_algorithm='bayesian')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,226.2107,90391.4240,300.6517,0.6727,0.2668,0.2131
1,230.9665,96681.7846,310.9369,0.6432,0.2778,0.2158
2,226.2027,88314.2872,297.1772,0.6564,0.2518,0.2005
3,223.5560,91074.7509,301.7859,0.6667,0.2639,0.2082
4,233.1827,97762.8488,312.6705,0.6323,0.2795,0.2243
5,235.4984,97997.0774,313.0448,0.6420,0.2707,0.2191
6,227.5048,92292.3553,303.7966,0.6526,0.2658,0.2129
7,235.2599,98574.5393,313.9658,0.6521,0.2723,0.2198
8,236.5196,103276.6757,321.3669,0.6475,0.2785,0.2208


Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [ ]:
# new_data = df.copy()
# new_data.drop('price', axis=1, inplace=True)
# new_data.head()

,location,bedroom,bathroom,land_area,building_area
0,"Rungkut, Surabaya",3,3,67,120
1,"Medokan Ayu, Surabaya",3,3,72,140
2,"Wonorejo, Surabaya",2,1,30,36
3,"Sukolilo, Surabaya",2,2,50,55
4,"Kebraon, Surabaya",2,1,50,56


In [ ]:
# predictions = predict_model(tuned_model, data = new_data)
# predictions.head()

,location,bedroom,bathroom,land_area,building_area,prediction_label
0,"Rungkut, Surabaya",3,3,67,120,1329.153096
1,"Medokan Ayu, Surabaya",3,3,72,140,1127.126952
2,"Wonorejo, Surabaya",2,1,30,36,412.174211
3,"Sukolilo, Surabaya",2,2,50,55,832.374166
4,"Kebraon, Surabaya",2,1,50,56,668.827038


In [14]:
# save_model(tuned_model, 'full-cb')